# imports

In [1]:
import pandas as pd
import numpy as np

# models
import xgboost as xgb

# data split
from sklearn.model_selection import train_test_split

# sklearn metrics
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score, recall_score


In [2]:
df = pd.read_csv('modeling_feats.csv',index_col= 0 )

# X and y split

In [3]:
# features
X = df.drop(columns= ['target','retained_job','chg_off_prin_gr'])
# Select dependent variable
y = df['target']

In [4]:
# checking X 
X.head(1)

,zip,term,no_emp,new_exist,create_job,franchise_code,urban_rural,rev_line_cr,low_doc,disbursement_gross,balance_gross,gr_appv,s_b_a__appv,sec_points,state_score
0,47711,84,4,2,0,1,0,0,1,60000,0,60000,48000,45,3


In [5]:
# checking y
y

0         0
1         0
2         0
3         0
4         0
         ..
899158    0
899160    0
899161    0
899162    1
899163    0
Name: target, Length: 611155, dtype: int64

# split the data

In [6]:
# split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [7]:
# different imports i might try
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.metrics import f1_score, make_scorer, confusion_matrix
from sklearn.ensemble import RandomForestClassifier


# scaling

In [8]:

from sklearn.preprocessing import StandardScaler
# creating a scaler instance
scaler = StandardScaler()

# fit and transforming to train
X_train_sc =scaler.fit_transform(X_train)

# transform to test
X_test_sc = scaler.transform(X_test)

In [9]:
# Creating dataframe from scaled data
X_train_sc = pd.DataFrame(X_train_sc)
X_test_sc = pd.DataFrame(X_test_sc)


# gridsearch

## random forest

In [16]:
# Create a random forest classifier
rfc = RandomForestClassifier()

# Define the parameter grid to search over
param_grid = {
    'n_estimators': [5,7,10],
    'max_depth': [3,5],
    'max_features': ['sqrt'],
    'min_samples_split': [2, 5, 7],
    'min_samples_leaf': [1, 3]
}

# Create a grid search object
grid_search = GridSearchCV(
    estimator=rfc,
    # using the dictionary
    param_grid=param_grid,
    # cross validate
    cv = 2,
    n_jobs = 2,
    # stop after 3 iteration with no improvment
    verbose= 3,
    # metric
    scoring='accuracy'
)


# Fit the grid search to the data scaled data
grid_search.fit(X_train_sc, y_train)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Fitting 2 folds for each of 36 candidates, totalling 72 fits
Best parameters:  {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 7, 'n_estimators': 10}
Best score:  0.887336273122203


# XGBoost

In [13]:
# create an object instances
model = xgb.XGBClassifier()

# fit model X_train_scaled, and y train
xgb_model = model.fit(X_train_sc, y_train)

In [14]:
# creating prediction based on X_test_scaled
y_hat = xgb_model.predict(X_test_sc)
# show accuracy score
print(f'Model XGboost Accuracy: {accuracy_score(y_test, y_hat):.2f}')

Model XGboost Accuracy: 0.95
